In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [2]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
### Load NER model
model_ner = spacy.load('./output/model-best/')

In [5]:
image = cv2.imread('./data/6.jpg')
cv2.imshow('businesscard',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# extract data using Pytesseract 
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

tessData = pytesseract.image_to_data(image)
tessData


'level\tpage_num\tblock_num\tpar_num\tline_num\tword_num\tleft\ttop\twidth\theight\tconf\ttext\n1\t1\t0\t0\t0\t0\t0\t0\t1065\t616\t-1\t\n2\t1\t1\t0\t0\t0\t3\t0\t1062\t5\t-1\t\n3\t1\t1\t1\t0\t0\t3\t0\t1062\t5\t-1\t\n4\t1\t1\t1\t1\t0\t3\t0\t1062\t5\t-1\t\n5\t1\t1\t1\t1\t1\t3\t0\t1062\t5\t95.000000\t \n2\t1\t2\t0\t0\t0\t47\t44\t8\t119\t-1\t\n3\t1\t2\t1\t0\t0\t47\t44\t8\t119\t-1\t\n4\t1\t2\t1\t1\t0\t47\t44\t8\t119\t-1\t\n5\t1\t2\t1\t1\t1\t47\t44\t8\t119\t95.000000\t \n2\t1\t3\t0\t0\t0\t46\t53\t979\t156\t-1\t\n3\t1\t3\t1\t0\t0\t722\t53\t303\t28\t-1\t\n4\t1\t3\t1\t1\t0\t722\t53\t303\t28\t-1\t\n5\t1\t3\t1\t1\t1\t722\t53\t64\t28\t90.167198\tCell\n5\t1\t3\t1\t1\t2\t802\t60\t5\t20\t90.167198\t:\n5\t1\t3\t1\t1\t3\t822\t53\t203\t28\t96.483879\t8099948528\n3\t1\t3\t2\t0\t0\t55\t55\t970\t112\t-1\t\n4\t1\t3\t2\t1\t0\t55\t55\t970\t89\t-1\t\n5\t1\t3\t2\t1\t1\t55\t55\t85\t89\t47.185547\t(Ga)\n5\t1\t3\t2\t1\t2\t822\t95\t203\t28\t96.435081\t8466045457\n4\t1\t3\t2\t2\t0\t593\t136\t432\t31\t-1\t\n5\t1\t3\t2

In [7]:
# Load Image
image = cv2.imread('./data/6.jpg')

# cv2.imshow('businesscard',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# extract data using Pytesseract 
tessData = pytesseract.image_to_data(image)
# convert into dataframe
tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True) # drop missing values
df['text'] = df['text'].apply(cleanText)

# convet data into content
df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)
# get prediction from NER model
doc = model_ner(content)

cell 8099948528 ga 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail “com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo


In [8]:
from spacy import displacy

In [9]:
# displacy.serve(doc,style='ent')

In [10]:
displacy.render(doc,style='ent')

### Tagging

In [11]:
doc.to_json()

{'text': 'cell 8099948528 ga 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail “com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo',
 'ents': [{'start': 5, 'end': 15, 'label': 'B-PHONE'},
  {'start': 19, 'end': 29, 'label': 'B-PHONE'},
  {'start': 36, 'end': 57, 'label': 'B-EMAIL'},
  {'start': 58, 'end': 62, 'label': 'B-ORG'},
  {'start': 63, 'end': 72, 'label': 'I-ORG'},
  {'start': 73, 'end': 84, 'label': 'I-ORG'},
  {'start': 85, 'end': 87, 'label': 'I-ORG'},
  {'start': 88, 'end': 93, 'label': 'I-ORG'},
  {'start': 94, 'end': 102, 'label': 'B-EMAIL'},
  {'start': 103, 'end': 113, 'label': 'I-EMAIL'},
  {'start': 114, 'end': 11

In [12]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [13]:
doc_text = docjson['text']
doc_text

'cell 8099948528 ga 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail “com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo'

In [14]:
docjson['tokens']

[{'id': 0, 'start': 0, 'end': 4},
 {'id': 1, 'start': 5, 'end': 15},
 {'id': 2, 'start': 16, 'end': 18},
 {'id': 3, 'start': 19, 'end': 29},
 {'id': 4, 'start': 30, 'end': 35},
 {'id': 5, 'start': 36, 'end': 57},
 {'id': 6, 'start': 58, 'end': 62},
 {'id': 7, 'start': 63, 'end': 72},
 {'id': 8, 'start': 73, 'end': 84},
 {'id': 9, 'start': 85, 'end': 87},
 {'id': 10, 'start': 88, 'end': 93},
 {'id': 11, 'start': 94, 'end': 102},
 {'id': 12, 'start': 103, 'end': 113},
 {'id': 13, 'start': 114, 'end': 115},
 {'id': 14, 'start': 115, 'end': 118},
 {'id': 15, 'start': 119, 'end': 129},
 {'id': 16, 'start': 130, 'end': 138},
 {'id': 17, 'start': 139, 'end': 148},
 {'id': 18, 'start': 149, 'end': 156},
 {'id': 19, 'start': 157, 'end': 162},
 {'id': 20, 'start': 163, 'end': 167},
 {'id': 21, 'start': 168, 'end': 170},
 {'id': 22, 'start': 170, 'end': 171},
 {'id': 23, 'start': 172, 'end': 180},
 {'id': 24, 'start': 181, 'end': 185},
 {'id': 25, 'start': 186, 'end': 195},
 {'id': 26, 'start': 1

In [15]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,4,cell
1,1,5,15,8099948528
2,2,16,18,ga
3,3,19,29,8466045457
4,4,30,35,email
5,5,36,57,lictsrikant@gmail.com
6,6,58,62,life
7,7,63,72,insurance
8,8,73,84,corporation
9,9,85,87,of


In [16]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [17]:
datafram_tokens.head(10)

,id,start,end,token,label
0,0,0,4,cell,NaN
1,1,5,15,8099948528,B-PHONE
2,2,16,18,ga,NaN
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,NaN
5,5,36,57,lictsrikant@gmail.com,B-EMAIL
6,6,58,62,life,B-ORG
7,7,63,72,insurance,I-ORG
8,8,73,84,corporation,I-ORG
9,9,85,87,of,I-ORG


In [18]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)

,id,start,end,token,label
0,0,0,4,cell,O
1,1,5,15,8099948528,B-PHONE
2,2,16,18,ga,O
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,O
5,5,36,57,lictsrikant@gmail.com,B-EMAIL
6,6,58,62,life,B-ORG
7,7,63,72,insurance,I-ORG
8,8,73,84,corporation,I-ORG
9,9,85,87,of,I-ORG


In [19]:
df_clean.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
12,5,1,3,1,1,1,722,53,64,28,90.167198,cell
14,5,1,3,1,1,3,822,53,203,28,96.483879,8099948528
17,5,1,3,2,1,1,55,55,85,89,47.185547,ga
18,5,1,3,2,1,2,822,95,203,28,96.435081,8466045457
20,5,1,3,2,2,1,593,136,93,25,89.289780,email


In [20]:
df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 

12      4
14     15
17     18
18     29
20     35
22     57
25     62
26     72
27     84
28     87
29     93
30    102
31    113
32    118
36    129
37    138
41    148
42    156
44    162
45    167
46    171
47    180
51    185
52    195
53    207
54    210
55    216
58    220
59    225
60    229
61    237
62    251
63    259
64    261
65    265
66    270
69    275
70    286
71    296
72    298
73    302
74    307
76    342
77    376
79    412
80    447
82    455
83    460
84    469
85    476
86    481
Name: text, dtype: int64

In [21]:
# join lable to df_clean dataframe
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

In [22]:
df_clean.head(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start
12,5,1,3,1,1,1,722,53,64,28,90.167198,cell,4,0
14,5,1,3,1,1,3,822,53,203,28,96.483879,8099948528,15,5
17,5,1,3,2,1,1,55,55,85,89,47.185547,ga,18,16
18,5,1,3,2,1,2,822,95,203,28,96.435081,8466045457,29,19
20,5,1,3,2,2,1,593,136,93,25,89.289780,email,35,30
22,5,1,3,2,2,3,709,136,316,31,86.249245,lictsrikant@gmail.com,57,36
25,5,1,3,3,1,1,46,170,33,14,96.406654,life,62,58
26,5,1,3,3,1,2,85,151,92,42,95.806709,insurance,72,63
27,5,1,3,3,1,3,183,170,117,14,96.909729,corporation,84,73
28,5,1,3,3,1,4,306,170,20,14,96.172005,of,87,85


In [23]:
# inner join with start 
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','token','label']],how='inner',on='start')

In [24]:
dataframe_info.tail(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
41,5,1,6,3,1,6,495,474,53,21,96.027008,018.,307,303,018,O
42,5,1,6,3,2,1,46,506,430,27,85.618164,"lictsrikant8099948528.blogspot.in,",342,308,lictsrikant8099948528.blogspot.in,I-WEB
43,5,1,6,3,2,2,488,506,429,27,89.912384,interviewsinhyderabad.blogspot.in,376,343,interviewsinhyderabad.blogspot.in,B-WEB
44,5,1,6,3,3,1,44,539,472,25,87.967415,"facebook.com/lictsrikant8099948528,",412,377,facebook.com/lictsrikant8099948528,O
45,5,1,6,3,3,2,526,539,443,22,90.545654,facebook.com/thathineni.srikanth.9,447,413,facebook.com/thathineni.srikanth.9,O
46,5,1,6,3,4,1,46,571,106,21,96.344551,promote,455,448,promote,O
47,5,1,6,3,4,2,161,576,56,22,95.835091,your,460,456,your,O
48,5,1,6,3,4,3,226,571,111,21,96.409111,business,469,461,business,O
49,5,1,6,3,4,4,347,571,74,21,93.162605,online,476,470,online,O
50,5,1,6,3,4,5,432,571,96,27,92.261208,pybo,481,477,pybo,O


### Bounding Box

In [25]:
bb_df = dataframe_info.query("label != 'O' ")
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,3,1,1,3,822,53,203,28,96.483879,8099948528,15,5,8099948528,B-PHONE
3,5,1,3,2,1,2,822,95,203,28,96.435081,8466045457,29,19,8466045457,B-PHONE
5,5,1,3,2,2,3,709,136,316,31,86.249245,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,B-EMAIL
6,5,1,3,3,1,1,46,170,33,14,96.406654,life,62,58,life,B-ORG
7,5,1,3,3,1,2,85,151,92,42,95.806709,insurance,72,63,insurance,I-ORG
8,5,1,3,3,1,3,183,170,117,14,96.909729,corporation,84,73,corporation,I-ORG
9,5,1,3,3,1,4,306,170,20,14,96.172005,of,87,85,of,I-ORG
10,5,1,3,3,1,5,332,170,42,14,96.796776,india,93,88,india,I-ORG
11,5,1,3,3,1,6,668,166,119,37,92.970848,seosrika,102,94,seosrika,B-EMAIL
12,5,1,3,3,1,7,791,165,162,44,91.918060,ntht@gmail,113,103,ntht@gmail,I-EMAIL


In [26]:
bb_df = dataframe_info.query("label != 'O' ")
img = image.copy()

for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Predictions',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,3,1,1,3,822,53,203,28,96.483879,8099948528,15,5,8099948528,PHONE
3,5,1,3,2,1,2,822,95,203,28,96.435081,8466045457,29,19,8466045457,PHONE
5,5,1,3,2,2,3,709,136,316,31,86.249245,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL
6,5,1,3,3,1,1,46,170,33,14,96.406654,life,62,58,life,ORG
7,5,1,3,3,1,2,85,151,92,42,95.806709,insurance,72,63,insurance,ORG


In [28]:
# group the label
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
        
    def getgroup(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id
        
grp_gen = groupgen()

In [29]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)

In [30]:
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label,group
1,5,1,3,1,1,3,822,53,203,28,96.483879,8099948528,15,5,8099948528,PHONE,1
3,5,1,3,2,1,2,822,95,203,28,96.435081,8466045457,29,19,8466045457,PHONE,1
5,5,1,3,2,2,3,709,136,316,31,86.249245,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL,2
6,5,1,3,3,1,1,46,170,33,14,96.406654,life,62,58,life,ORG,3
7,5,1,3,3,1,2,85,151,92,42,95.806709,insurance,72,63,insurance,ORG,3
8,5,1,3,3,1,3,183,170,117,14,96.909729,corporation,84,73,corporation,ORG,3
9,5,1,3,3,1,4,306,170,20,14,96.172005,of,87,85,of,ORG,3
10,5,1,3,3,1,5,332,170,42,14,96.796776,india,93,88,india,ORG,3
11,5,1,3,3,1,6,668,166,119,37,92.970848,seosrika,102,94,seosrika,EMAIL,4
12,5,1,3,3,1,7,791,165,162,44,91.918060,ntht@gmail,113,103,ntht@gmail,EMAIL,4


In [31]:
# right and bottom of bounding box
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

In [32]:
# tagging: groupby group
col_group = ['left','top','right','bottom','label','token','group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [33]:
img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'token':lambda x: " ".join(x)
    
})

In [34]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,822,1025,53,123,[PHONE],8099948528 8466045457
2,709,1025,136,167,[EMAIL],lictsrikant@gmail.com
3,46,374,151,193,[ORG],life insurance corporation of india
4,668,1025,165,209,[EMAIL],seosrika ntht@gmail “
5,310,755,227,259,[NAME],thathineni srikanth
6,399,669,271,296,[DES],insurance advisor
7,47,882,395,427,[ORG],life insurance corporation of india
8,46,917,506,533,[WEB],lictsrikant8099948528.blogspot.in interviewsin...


In [35]:
img_bb = image.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb,(l,t),(r,b),(0,255,0),2)
    
    cv2.putText(img_bb, str(label) ,(l,t),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Bounding Box BusinessCard',img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Parser


In [36]:
def parser(text, label):
    if label == 'PHONE':
        text = text.lower()
        text = re.sub(r'\D','',text)

    elif label == 'EMAIL':
        text = text.lower()
        allow_special_char = '@_.\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)

    elif label == 'WEB':
        text = text.lower()
        allow_special_char = ':/.%#\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)

    elif label in ('NAME', 'DES'):
        text = text.lower()
        text = re.sub(r'[^a-z ]','',text)
        text = text.title()

    elif label == 'ORG':
        text = text.lower()
        text = re.sub(r'[^a-z0-9 ]','',text)
        text = text.title()

    return text

In [37]:
parser('madhav y84327', 'NAME')

'Madhav Y'

In [38]:
parser('madhav%67fd@gmail.com', 'EMAIL')

'madhav67fd@gmail.com'

### Entity

In [39]:
info_array = dataframe_info[['token','label']].values
entities = dict(NAME=[],ORG=[],DES=[],PHONE=[],EMAIL=[],WEB=[])
previous = 'O'

for token, label in info_array:
    bio_tag = label[0]
    label_tag = label[2:]
    
    # step -1 parse the token
    text = parser(token,label_tag)
    
    if bio_tag in ('B','I'):
        
        if previous != label_tag:
            entities[label_tag].append(text)
            
        else:
            if bio_tag == "B":
                entities[label_tag].append(text)
                
            else:
                if label_tag in ("NAME",'ORG','DES'):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
                    
                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + text
                    
    
    
    previous = label_tag


In [40]:
entities

{'NAME': ['Thathineni Srikanth'],
 'ORG': ['Life Insurance Corporation Of India',
  'Life Insurance Corporation Of India'],
 'DES': ['Insurance Advisor'],
 'PHONE': ['8099948528', '8466045457'],
 'EMAIL': ['lictsrikant@gmail.com', 'seosrikantht@gmail'],
 'WEB': ['lictsrikant8099948528.blogspot.in',
  'interviewsinhyderabad.blogspot.in']}